# SmolVLAモデルの訓練

## 概要

作成したデータセットを使用し、[SmolVLAのベースモデル][1]をファインチューニングする。

[1]: https://huggingface.co/docs/lerobot/en/smolvla
[1]: https://github.com/hayatoshibahara/smolvla

## 環境構築

In [ ]:
# 環境変数の取得

try: 
    # Google Colabの場合
    from google.colab import userdata
    IS_COLAB = True
    HF_USER = userdata.get("HF_USER")
    HF_TOKEN = userdata.get("HF_TOKEN")
    WANDB_API_KEY = userdata.get("WANDB_API_KEY")
except:
    # ローカル環境の場合
    from dotenv import load_dotenv
    IS_COLAB = False
    import os
    load_dotenv()
    HF_USER = os.getenv("HF_USER")
    HF_TOKEN = os.getenv("HF_TOKEN")
    WANDB_API_KEY = os.getenv("WANDB_API_KEY")

assert HF_USER
assert HF_TOKEN
assert WANDB_API_KEY

In [ ]:
# Conda環境のセットアップ

try:
    import condacolab
except:
    %pip install -q condacolab
    import condacolab
    condacolab.install()

In [ ]:
# Wandbのログイン

import wandb
wandb.login(key=WANDB_API_KEY)

In [ ]:
## LeRobotのインストール

if IS_COLAB:
    import os
    if not os.path.exists("/content/lerobot"):
        !git clone -b v0.4.2 --depth 1 https://github.com/huggingface/lerobot.git
    %cd /content/lerobot
    %pip install -e .
    %pip install -e ".[smolvla]"
else:
    import os
    if not os.path.exists("/workspace/lerobot"):
        !git clone -b v0.4.2 --depth 1 https://github.com/huggingface/lerobot.git
    %cd /workspace/lerobot
    %pip install -e .
    %pip install -e ".[smolvla]"

import lerobot
%pip show lerobot

# 初回はここで再起動が必要

## 訓練

In [ ]:
from datetime import datetime
now = datetime.now()
DATETIME = now.strftime("%Y-%m-%d-%H-%M-%S")

# 使用するデータセット名
DATASET_REPO_NAME = "your-dataset-repo-name"

OUTPUT_DIR = f"{DATASET_REPO_NAME}_smolvla_{DATETIME}"
JOB_NAME = f"{DATASET_REPO_NAME}_smolvla_{DATETIME}"
POLICY_REPO_NAME = f"{DATASET_REPO_NAME}_smolvla_{DATETIME}"

# バッチサイズ
# メモリ不足が発生する場合は下げ、逆に余裕がある場合は上げる
# 1, 2, 4, 8, 16...など2の累乗が一般的
# A100の場合は8、T4の場合は4が目安
BATCH_SIZE = 64

COMMAND = f"""
lerobot-train
    --batch_size={BATCH_SIZE}
    --job_name={JOB_NAME}
    --num_workers=0
    --resume=false
    --save_freq=1000
    --steps=20000
    --output_dir={OUTPUT_DIR}
    --dataset.repo_id={HF_USER}/{DATASET_REPO_NAME}
    --policy.path=lerobot/smolvla_base
    --policy.device=cuda
    --policy.repo_id={HF_USER}/{POLICY_REPO_NAME}
    --wandb.enable=true
"""

print(COMMAND)

# コマンドを実行
!$COMMAND

# 注意: VLAは画像特徴量の名前が予め決まっている
# - observation.images.camera1
# - observation.images.camera2
# - observation.images.camera3
# データセットの画像特徴量の名前が異なる場合は、以下のようにリネームマップを作成する
# RENAME_MAP = '{"observation.images.wrist.images": "observation.images.camera1", "observation.images.top.images": "observation.images.camera2"}'
# そして、上記のコマンドに以下のオプションを追加する
# --rename_map='{RENAME_MAP}'

コマンドライン引数の説明は、`src/lerobot/configs/train.py`にある。

一般的な設定

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `dataset` | `DatasetConfig` | (必須) | データセット設定。 |
| `env` | `envs.EnvConfig \| None` | `None` | 環境設定。 |
| `policy` | `PreTrainedConfig \| None` | `None` | ポリシー設定。 |
| `output_dir` | `Path \| None` | `None` | 実行時の全出力を保存するディレクトリ。<br>`resume`を`True`に設定しない限り、同じディレクトリで別のトレーニングセッションを実行すると、内容は上書きされます。 |
| `job_name` | `str \| None` | `None` | ジョブ名。 |
| `resume` | `bool` | `False` | 以前の実行を再開するかどうか。<br>再開するには、`output_dir`が少なくとも1つのチェックポイントを含む既存の実行ディレクトリである必要があります。<br>再開時、コマンドライン引数に関わらず、チェックポイント内の設定がデフォルトで使用されることに注意してください。 |
| `seed` | `int \| None` | `1000` | トレーニング（モデル初期化、データセットのシャッフルなど）および評価環境で使用されるシード値。 |
| `num_workers` | `int` | `4` | データローダーで使用するワーカー数。 |
| `batch_size` | `int` | `8` | バッチサイズ。 |
| `steps` | `int` | `100_000` | トレーニングを行う総ステップ数。 |
| `eval_freq` | `int` | `20_000` | 評価を実行する頻度（ステップ数）。 |
| `log_freq` | `int` | `200` | ログを記録する頻度（ステップ数）。 |
| `save_checkpoint` | `bool` | `True` | チェックポイントを保存するかどうか。 |
| `save_freq` | `int` | `20_000` | チェックポイントを保存する頻度（ステップ数）。最後のトレーニングステップ後にも保存されます。 |
| `use_policy_training_preset` | `bool` | `True` | ポリシー固有のトレーニングプリセット（推奨されるオプティマイザ設定など）を使用するかどうか。 |
| `optimizer` | `OptimizerConfig \| None` | `None` | オプティマイザ設定。 |
| `scheduler` | `LRSchedulerConfig \| None` | `None` | 学習率スケジューラ設定。 |
| `eval` | `EvalConfig` | `EvalConfig()` | 評価設定。 |
| `wandb` | `WandBConfig` | `WandBConfig()` | Weights & Biasesの設定。 |
| `checkpoint_path` | `Path \| None` | `None` | （内部使用）再開時に使用されるチェックポイントのパス。初期化時は `None`。 |
| `rename_map` | `dict[str, str]` | `{}` | 観測データ（画像や状態のキー）を上書きするためのリネームマップ。 |

データセットの設定（DatasetConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `repo_id` | `str` | (必須) | データセットのリポジトリID。データセットのリストを指定することもでき、その場合 `train.py` がそれらを連結します。<br>注意: データセット間で共通するデータキーのみが保持されます。各データセットには、返されるアイテムに "dataset_index" を挿入する追加の変換処理が適用され、インデックスの割り当てはデータセットの指定順に基づいて行われます。 |
| `root` | `str \| None` | `None` | データセットが保存されるルートディレクトリ（例: `dataset/path`）。 |
| `episodes` | `list[int] \| None` | `None` | 使用するエピソードのリスト。 |
| `image_transforms` | `ImageTransformsConfig` | `ImageTransformsConfig()` | 画像変換設定。 |
| `revision` | `str \| None` | `None` | データセットのリビジョン（バージョン）。 |
| `use_imagenet_stats` | `bool` | `True` | ImageNetの統計情報（平均・標準偏差）を使用するかどうか。 |
| `video_backend` | `str` | *システム依存* | 動画の読み込みに使用するバックエンド（例: `pyav`, `opencv`）。 |
| `streaming` | `bool` | `False` | ストリーミングモード（データを全てダウンロードせずに逐次読み込む）を使用するかどうか。 |

環境設定（EnvConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `task` | `str \| None` | `None` | タスクの名称。 |
| `fps` | `int` | `30` | 1秒あたりのフレーム数（FPS）。 |
| `features` | `dict[str, PolicyFeature]` | `{}` | 環境が提供する特徴量（観測やアクションなど）の定義。 |
| `features_map` | `dict[str, str]` | `{}` | 特徴量の名前マッピング。 |
| `max_parallel_tasks` | `int` | `1` | 並行して実行するタスクの最大数。 |
| `disable_env_checker` | `bool` | `True` | Gymの環境チェッカーを無効にするかどうか。 |

事前学習済みモデルの設定（PreTrainedConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `n_obs_steps` | `int` | `1` | ポリシーに渡す観測データの環境ステップ数（現在のステップと、過去に遡る追加ステップを含みます）。 |
| `input_features` | `dict[str, PolicyFeature]` | `{}` | ポリシーへの入力データの定義。 |
| `output_features` | `dict[str, PolicyFeature]` | `{}` | ポリシーからの出力データの定義。 |
| `device` | `str \| None` | `None` | 使用するデバイス（例: "cuda", "cuda:0", "cpu", "mps"）。 |
| `use_amp` | `bool` | `False` | トレーニングおよび評価でAutomatic Mixed Precision (AMP)を使用するかどうかを決定します。AMPを使用すると、自動勾配スケーリングが有効になります。 |
| `push_to_hub` | `bool` | `True` | モデルをHugging Face Hubにアップロードするかどうか。 |
| `repo_id` | `str \| None` | `None` | Hugging Face HubのリポジトリID。 |
| `private` | `bool \| None` | `None` | Hugging Face Hubのプライベートリポジトリにアップロードするかどうか。 |
| `tags` | `list[str] \| None` | `None` | Hub上のポリシーに追加するタグのリスト。 |
| license | `str \| None` | `None` | ライセンス情報。 |
| `pretrained_path` | `Path \| None` | `None` | HubでホストされているモデルのリポジトリID、または `Policy.save_pretrained` を使用して保存された重みを含むディレクトリへのパス。<br>指定されない場合、ポリシーはスクラッチから初期化されます。 |

オプティマイザの設定（OptimizerConfig）

| フィールド名 | 型 | 説明 |
| :--- | :--- | :--- |
| `lr` | `float` | 学習率（Learning Rate）。 |
| `weight_decay` | `float` | 重み減衰（Weight Decay）。過学習を防ぐための正則化項です。 |
| `grad_clip_norm` | `float` | 勾配クリッピングのノルム値。勾配爆発を防ぐために勾配のノルムをこの値に制限します。 |

学習スケージューラの設定（LRSchedulerConfig）

| フィールド名 | 型 | 説明 |
| :--- | :--- | :--- |
| `num_warmup_steps` | `int` | ウォームアップステップ数。学習率を0から初期学習率まで徐々に上げていく期間のステップ数です。 |

Weight and Biases の設定（WandbConfig）

| フィールド名 | 型 | デフォルト値 | 説明 |
| :--- | :--- | :--- | :--- |
| `enable` | `bool` | `False` | Weights & Biases (WandB) のログ記録を有効にするかどうか。 |
| `disable_artifact` | `bool` | `False` | `training.save_checkpoint=True` であっても、アーティファクトの保存を無効にする場合は `True` に設定します。 |
| `project` | `str` | `"lerobot"` | WandBのプロジェクト名。 |
| `entity` | `str \| None` | `None` | WandBのエンティティ（ユーザー名またはチーム名）。 |
| `notes` | `str \| None` | `None` | 実行に関するメモ。 |
| `run_id` | `str \| None` | `None` | 実行ID。以前の実行を特定して再開する場合などに使用します。 |
| `mode` | `str \| None` | `None` | 実行モード（許可される値: `'online'`, `'offline'`, `'disabled'`）。デフォルトは `'online'` です。 |

## トラッキング

Wandbで訓練損失をトラッキングできる。

損失の変動が落ち着くまで訓練を続ける（以下の場合は20kで早期終了する）。

## 非同期アップロード

Google Colabでターミナルを開き、訓練中のモデルをアップロードできる。

```sh
export HF_USER="..."
export HF_TOKEN="..."
export DATETIME=$(date +"%Y-%m-%d-%H-%M-%S")
export NUM_STEPS=2000
export MODEL_REPO_NAME="grab-a-cube_smolvla_${DATETIME}_${NUM_STEPS}"

hf upload \
    --token $HF_TOKEN \
    $HF_USER/$MODEL_REPO_NAME \
    lerobot/your_model_name/checkpoints/002000/pretrained_model
```